# Problema do Empacotamento

Problema bastante importante na área de logistica, mais especficamente na área de armazenagem.

Objetivo do problema é gaurdar todos os itens em caixas minimizando a quantidade de caixas necessárias.

**Observação** Os itens não podem ser fracionados.

Índices:
- i = 1, ..., m itens;
- j = 1, ..., n caixas;

Parâmetroas:
- $w_i$ peso do item i;
- $C$ capacidade das caixas;

Limitante inferior = $\left[\frac{\sum_{i=1}^{n}w_i}{C}\right]$. Corresponde uma estimativa de quanto que é o melhor caso possível para solução ótima.

Limitante superior obtém pelo algoritmo ``next fit``.


Variáveis de Decisão (todas binárias)
- $x_{ij}$ indica se o item i será colocado na caixa j;
- $y_j$ indica se a caixa j foi usada na solução;

#### Função Objetivo

$$
\min\left[ \sum_{j=1}^{n}y_j \right]
$$

#### Função Restrição:

restrição na capacidade da caixa
$$
\sum_{i=1}^{m}w_ix_{ij} \leq C y_{j}, \;\; \forall j=1, ..., n
$$

todos os itens devem estar em uma caixa
$$
\sum_{j=1}^{n}x_{ij} = 1, \;\; \forall i=1, ..., m
$$

In [1]:
import gurobipy as gp

In [3]:
# Dados do problema 
qtd_itens = 15    ## Quantidades de itens
qtd_caixas = 9    ## Quantidades de caixas inicialmente que pode ser obtida pelo algoritmo ``next fit``.
capacidade = 100  ## Capacidade de cada caixa

# lista com os pesos de cada item
vet_pesos = [20, 35, 60, 10, 30, 45, 30, 65, 60, 30, 50, 20, 65, 70, 45]

# Rótulos dos itens
itens = [f'Item_{i}' for i in range(1,qtd_itens+1)]

# Rótulos das caixas
caixas = [f'Caixa_{j}' for j in range(1,qtd_caixas+1)]

# Dicionário dos pesos
pesos = {itens[i]:vet_pesos[i] for i in range(qtd_itens)}

In [8]:
# Modelo
m = gp.Model()

# Variáveis de decisão
x = m.addVars(itens, caixas, vtype=gp.GRB.BINARY)
y = m.addVars(caixas, vtype=gp.GRB.BINARY)

# Função Objetivo
m.setObjective(
    gp.quicksum(y[j] for j in caixas),
    sense=gp.GRB.MINIMIZE
)

# Restrição de todos os itens em uma caixa
c1 = m.addConstrs(
    gp.quicksum(x[i,j] for j in caixas) == 1 for i in itens
)

# Restrição de capacidade
c2 = m.addConstrs(
    gp.quicksum(pesos[i]*x[i,j] for i in itens) <= capacidade*y[j] for j in caixas
)

# Executar o modelo
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (linux64)
Optimize a model with 24 rows, 144 columns and 279 nonzeros
Model fingerprint: 0xe95c5229
Variable types: 0 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 9.0000000
Presolve time: 0.04s
Presolved: 24 rows, 144 columns, 279 nonzeros
Variable types: 0 continuous, 144 integer (144 binary)

Root relaxation: objective 6.350000e+00, 41 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.35000    0    9    9.00000    6.35000  29.4%     -    0s
H    0     0                       7.0000000    6.35000  9.29%     -    0s
     0     0    6.35000    0    9    7.00000    6.35000  9.29%     -    0s

Explored 1 nodes (41 s

In [12]:
# Qual item está em qual caixa?
for caixa in caixas:
    print(caixa)
    for item in itens:
        if round(x[item,caixa].X) == 1:
            print(item, ' ', end='')
    print('\n')

Caixa_1
Item_10  Item_13  

Caixa_2
Item_12  Item_15  

Caixa_3
Item_3  Item_7  

Caixa_4


Caixa_5
Item_6  Item_11  

Caixa_6
Item_1  Item_14  

Caixa_7
Item_2  Item_8  

Caixa_8
Item_4  Item_5  Item_9  

Caixa_9


